# Check Conda Env Name

In [1]:
import sys
sys.executable.split('/')[-3]

'pytor'

# Result Filename

In [2]:
result_filename='Results_FOM.csv'

# Imports

In [3]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.face import FaceClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
from array import array
import os
import sys
import time
from PIL import Image
import csv
from time import sleep
from glob import glob

## API Setting 

In [4]:
subscription_key = "4d673dca4fbb43ab8906a7ae5691c866"
endpoint="https://advdf.cognitiveservices.azure.com/"
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))
subscription_key = "2b4d12d9d44940bbbd8c48c8f3c0e575"
endpoint="https://advdf-similarity.cognitiveservices.azure.com/"
face_client=FaceClient(endpoint, CognitiveServicesCredentials(subscription_key))

# FOM

### Describe an Image

In [5]:
'''
Describe an Image - local
This example describes the contents of an image with the confidence score.'Scarlett Johansson_fake.png'
'''
def describe_image(filename):
#     print("===== Describe an image - local =====")
    # Call API
    image= open(filename,"rb")
    description_results = computervision_client.describe_image_in_stream(image)

    # Get the captions (descriptions) from the response, with confidence level
#     print("Description of local image: ")
    if (len(description_results.captions) == 0):
#         print("No description detected.")
        return "No description detected."
    else:
        for caption in description_results.captions:
#             print("'{}' with confidence {:.2f}%".format(caption.text, caption.confidence * 100))
            return "'{}' with confidence {:.2f}%".format(caption.text, caption.confidence * 100)

### Detect Domain-specific Content

In [6]:
'''
Detect Domain-specific Content - local
This example detects celebrites and landmarks in local images.
'''
def domain_specific_content(filename):
#     print("===== Detect Domain-specific Content - local =====")
    # Image of one or more celebrities
    image= open(filename,"rb")
    # image = image.read('Scarlett.png')

    # Call API with content type (celebrities) and URL
    detect_domain_results_celebs_remote = computervision_client.analyze_image_by_domain_in_stream("celebrities", image)
    # Print detection results with name
#     print("Celebrities in the local image:")
    if len(detect_domain_results_celebs_remote.result["celebrities"]) == 0:
        print("No celebrities detected.")
        return "",""
    else:
        for celeb in detect_domain_results_celebs_remote.result["celebrities"]:
#             print(celeb["name"])
#             print(celeb["confidence"])
            return celeb["name"],celeb["confidence"]

In [7]:
def get_name(filename):
    return os.path.splitext(os.path.basename(filename))[0].split("_")[0]

In [8]:
def check_similarity(src_image, tar_image):
    # Detect a face in an image that contains a single face
    src_image_name = os.path.basename(src_image)
    detected_faces = face_client.face.detect_with_stream(open(src_image,"rb"))
    if not detected_faces:
        print('No face detected from image {}'.format(src_image_name))
        return 0
#         raise Exception('No face detected from image {}'.format(single_image_name))

    # Display the detected face ID in the first single-face image.
    # Face IDs are used for comparison to faces (their IDs) detected in other images.
#     print('Detected face ID from', single_image_name, ':')
#     for face in detected_faces: 
#         print (face.face_id)
#     print()

    # Save this ID for use in Find Similar
    first_image_face_ID = detected_faces[0].face_id
    
    tar_image_name = os.path.basename(tar_image)
    detected_faces2 = face_client.face.detect_with_stream(open(tar_image,"rb"))
    if not detected_faces2:
        print('No face detected from image {}'.format(tar_image_name))
        return 0
#         raise Exception('No face detected from image {}'.format(multi_image_name))
    
    # Search through faces detected in group image for the single face from first image.
    # First, create a list of the face IDs found in the second image.
    second_image_face_IDs = list(map(lambda x: x.face_id, detected_faces2))
    # Next, find similar face IDs like the one detected in the first image.
    similar_faces = face_client.face.find_similar(face_id=first_image_face_ID, face_ids=second_image_face_IDs)
    
    if not similar_faces:
        print('No similar faces found in', tar_image_name, '.')
        return 0
    else:
        # Print the details of the similar faces detected
#         print('Similar faces found in', multi_image_name + ':')
        for face in similar_faces:
        #     print(face)
            return face.confidence
#             print('  Confidence:', face.confidence)
#             first_image_face_ID = face.face_id
#             # The similar face IDs of the single face image and the group image do not need to match, 
#             # they are only used for identification purposes in each image.
#             # The similar faces are matched using the Cognitive Services algorithm in find_similar().
#             face_info = next(x for x in detected_faces2 if x.face_id == first_image_face_ID)
#             if face_info:
#                 print('  Face ID: ', first_image_face_ID)
#                 print('  Face rectangle:')
#                 print('    Left: ', str(face_info.face_rectangle.left))
#                 print('    Top: ', str(face_info.face_rectangle.top))
#                 print('    Width: ', str(face_info.face_rectangle.width))
#                 print('    Height: ', str(face_info.face_rectangle.height))

In [9]:
file=open(result_filename, 'w', newline='')
writer = csv.writer(file)
writer.writerow(["Filename", "Src","Tar", "Src Pred.", "Src Conf.", "Tar Pred.","Tar Conf.", "Similarity","Src Des", "Tar Des"])
file.close()
actors_list=glob("/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/*")
count=0
for actor in actors_list:
    count+=1
    videos_list=glob(os.path.join(actor,"*"))
    for video in videos_list:
        print(video)
        images_list=glob(os.path.join(video,"*"))
        if len(images_list)<2:
            continue
        fake_images_list=[x for x in images_list if "_fake.png" in x]
        real_image=[x for x in images_list if "_real.png" in x][0]
        print(real_image)
        result_real_pred,result_real_conf=domain_specific_content(real_image)
        real_des=describe_image(real_image)
        src=get_name(real_image)
        for fake_image in fake_images_list:
            print(fake_image)
#             continue
            result_fake_pred,result_fake_conf=domain_specific_content(fake_image)
            fake_des=describe_image(fake_image)
            tar=get_name(fake_image)
            similarity=check_similarity(real_image,fake_image)
#             print(fake_image,src,tar,
#                   result_real_pred,result_real_conf,
#                   result_fake_pred,result_fake_conf,
#                   similarity,real_des,fake_des)
            file=open(result_filename, 'a', newline='')
            writer = csv.writer(file)
            writer.writerow([fake_image,src,tar,
                  result_real_pred,result_real_conf,
                  result_fake_pred,result_fake_conf,
                  similarity,real_des,fake_des])
            file.close()
#             sleep(10)
#             break
#         break
#     break
# file.close()
print("Total:", count)

/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Emma Watson/id11_0009
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Emma Watson/id11_0009/Emma Watson_real.png
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Emma Watson/id11_0009/Gal Gadot_fake.png
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Emma Watson/id11_0009/Liam Neeson_fake.png
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Emma Watson/id11_0009/Jessica Alba_fake.png
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Emma Watson/id11_0009/Eva Mendes_fake.png
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Emma Watson/id11_0010
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Emma Watson/id11_0010/Emma Watson_real.png
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Emma Watson/id11_0010/Keira Knightley_fake.png
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-T

No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Megan Fox/id46_0003
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Megan Fox/id46_0003/Megan Fox_real.png
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Megan Fox/id46_0003/Gal Gadot_fake.png
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Megan Fox/id46_0003/Jim Carrey_fake.png
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Megan Fox/id46_0003/Natalie Portman_fake.png
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Megan Fox/id46_0003/Angelina Jolie_fake.png
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Megan Fox/id46_0006
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Megan Fox/id46_0006/Megan Fox_real.png
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Megan Fox/id46_0006/Denzel Washington_fake.pn

/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Russell Crowe/id32_0000/Emma Stone_fake.png
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Russell Crowe/id32_0000/Colin Farrell_fake.png
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Russell Crowe/id32_0007
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Russell Crowe/id32_0007/Russell Crowe_real.png
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Russell Crowe/id32_0007/Jake Gyllenhaal_fake.png
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Russell Crowe/id32_0007/Jennifer Aniston_fake.png
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Russell Crowe/id32_0007/Jason Statham_fake.png
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Russell Crowe/id32_0007/Cameron Diaz_fake.png
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Russell Crowe/id32_0005
/ho

No celebrities detected.
No similar faces found in Jake Gyllenhaal_fake.png .
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Randall Park/id9_0005/Jennifer Aniston_fake.png
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Randall Park/id9_0005/Brad Pitt_fake.png
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Randall Park/id9_0001
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Randall Park/id9_0001/Randall Park_real.png
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Randall Park/id9_0001/Chris Pine_fake.png
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Randall Park/id9_0001/Emma Watson_fake.png
No celebrities detected.
No similar faces found in Emma Watson_fake.png .
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Randall Park/id9_0001/Leonardo DiCaprio_fake.png
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Randal

/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Emma Stone/id48_0003/Scarlett Johansson_fake.png
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Emma Stone/id48_0003/Edward Norton_fake.png
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Emma Stone/id48_0003/Chris Hemsworth_fake.png
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Emma Stone/id48_0009
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Emma Stone/id48_0009/Emma Stone_real.png
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Emma Stone/id48_0009/Tom Hanks_fake.png
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Emma Stone/id48_0009/Ryan Gosling_fake.png
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Emma Stone/id48_0009/Evangeline Lilly_fake.png
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Emma Stone/id48_0009/Kate

...

No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Jessica Alba/id43_0003/Justin Timberlake_fake.png
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Jessica Alba/id43_0003/Leonardo DiCaprio_fake.png
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Jessica Alba/id43_0003/Megan Fox_fake.png
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Jessica Alba/id43_0001
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Jessica Alba/id43_0001/Jessica Alba_real.png
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Jessica Alba/id43_0001/Tom Hanks_fake.png
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Jessica Alba/id43_0001/Ben Affleck_fake.png
No celebrities detected.
No similar faces found in Ben Affleck_fake.png .
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Jessica Alba/id43_0001/Coli

/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Amir Khan/id0_0006/Jim Carrey_fake.png
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Amir Khan/id0_0006/Anne Hathaway_fake.png
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Amir Khan/id0_0006/Famke Janssen_fake.png
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Amir Khan/id0_0006/John Travolta_fake.png
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Amir Khan/id0_0004
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Amir Khan/id0_0004/Amir Khan_real.png
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Amir Khan/id0_0004/Carrie-Anne Moss_fake.png
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Amir Khan/id0_0004/Edward Norton_fake.png
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Amir Khan/id0_0004/Jennifer Aniston_fake.png
/home/sha/project/adversarial_deepfake/src/Similarit

No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Carrie-Anne Moss/id57_0008/Keira Knightley_fake.png
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Carrie-Anne Moss/id57_0008/Chloe Grace Moretz_fake.png
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Carrie-Anne Moss/id57_0008/Don Cheadle_fake.png
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Carrie-Anne Moss/id57_0008/John Travolta_fake.png
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Carrie-Anne Moss/id57_0000
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Carrie-Anne Moss/id57_0000/Carrie-Anne Moss_real.png
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Carrie-Anne Moss/id57_0000/Yang Lan_fake.png
No celebrities detected.
/home/sha/project/adversarial_dee

No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Angelina Jolie/id13_0009/Leonardo DiCaprio_fake.png
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Angelina Jolie/id13_0009/Ethan Hawke_fake.png
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Angelina Jolie/id13_0010
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Angelina Jolie/id13_0010/Angelina Jolie_real.png
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Angelina Jolie/id13_0010/Chloe Grace Moretz_fake.png
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Angelina Jolie/id13_0010/Leonardo DiCaprio_fake.png
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Angelina Jolie/id13_0010/Chris Evans_fake.png
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Angelina Jolie/id13_0010/John Travolta_fake.png
N

No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Angelina Jolie/id13_0005/Liam Neeson_fake.png
No celebrities detected.
/home/sha/project/adversarial_deepfake/src/Similarity/FOM-Talking/Angelina Jolie/id13_0005/Will Smith_fake.png
No celebrities detected.
No similar faces found in Will Smith_fake.png .
Total: 58
